In [1]:
%gui qt

In [2]:
import numpy as np
from vispy import scene

from PyQt5 import QtCore
from PyQt5.QtCore import Qt, QThread, QEventLoop
from PyQt5.QtWidgets import QMainWindow, QAction, QFileDialog, QWidget, QSplitter, QComboBox, QTextBrowser, QSlider, QPushButton, QTableWidget, QVBoxLayout, QHBoxLayout, QLineEdit, QLabel, QGridLayout
from PyQt5.QtGui import QIcon
from PyQt5.QtWidgets import QApplication


class View(scene.SceneCanvas):
    def __init__(self):
        super().__init__(keys=None)

        self.unfreeze()

        self.grid = self.central_widget.add_grid(margin=10)
        self.view = self.grid.add_view(row=0, col=1, border_color='white')

        self.view.camera = 'panzoom'
        self.line = None
        self.add_axis()
        self.freeze()

    def add_axis(self, axis_color=(0,1,1,0.8)):
        fg = axis_color

        self.unfreeze()
        self.yaxis = scene.AxisWidget(orientation='left', text_color=fg,
                                      axis_color=fg, tick_color=fg)
        self.yaxis.stretch = (0.1, 0.9)
        self.grid.add_widget(self.yaxis, row=0, col=0)


        self.xaxis = scene.AxisWidget(orientation='bottom', text_color=fg,
                                      axis_color=fg, tick_color=fg)
        self.xaxis.stretch = (0.9, 0.1)
        self.grid.add_widget(self.xaxis, row=1, col=1)

        self.xaxis.link_view(self.view)
        self.yaxis.link_view(self.view)

        self.freeze()
    
    def set_data(self, pos):
        self.clear()
        self.line = scene.visuals.Line(pos=pos, color='white', width=2)
        self.view.add(self.line)
        self.view.camera.set_range()
    
    def clear(self):
        if self.line is not None:
            self.line.parent = None
            self.line = None

    # def attach(self, gui):
    #     self.unfreeze()
    #     gui.add_view(self)

    # def on_key_press(self, e):
    #     if e.text == 'r':
    #         self.view.camera.reset()
    #         self.view.camera.set_range()


class GUI(QWidget):
    def __init__(self):
        super().__init__()

        self.plot = View()
        self.timer = QtCore.QTimer()
        self.timer.timeout.connect(self.update)

        self.btn = QPushButton('Start')
        self.btn.setCheckable(True)
        self.btn.toggled.connect(self.start)

        layout_main = QVBoxLayout()
        layout_main.addWidget(self.btn)
        layout_main.addWidget(self.plot.native)
        self.setLayout(layout_main)
        self.data = None
        self.time = 0

    def set_data(self, pos):
        self.data = pos
    
    def update(self):
        self.plot.set_data(self.data[self.time:self.time+1000])
        self.time += 1

    def start(self, state):
        if state:
            self.timer.start(200)
        else:
            self.timer.stop()

In [3]:
pos_x = np.arange(100000)
pos_y = np.random.randn(100000)

gui = GUI()
gui.set_data(np.stack([pos_x, pos_y], axis=1))
gui.show()